# COURSERA CAPSTONE PROJECT: Week 3 Assignment

## SECTION A

###  QUESTION 1: Start by creating a new Notebook for this assignment.
###  QUESTION 2: Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [ ]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
#import html5lib
from pandas.io.html import read_html

In [35]:
#Extracting table from Wiki page.
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={"class":"wikitable"})
print("Extracted {num} wikitables".format(num=len(wikitables)))
NBH_df = wikitables[0]

Extracted 1 wikitables


### QUESTION 3: Create Dataframe

In [36]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
NBH_df = NBH_df[NBH_df.Borough != "Not assigned"]

In [37]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
NBH_df['temp_column'] = np.where(NBH_df['Neighbourhood']=='Not assigned',NBH_df['Borough'],NBH_df['Neighbourhood'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
# More than one neighborhood can exist in one postal code area.Neighbourhoods in thesame postal codes are grouped together in a row
NBH_df = NBH_df.groupby(['Postcode','Borough'])['temp_column'].apply(', '.join).reset_index()
NBH_df.head(3)
# Rename the columns as indicated in the instructions
NBH_df = NBH_df.rename(columns = {"Postcode": "PostalCode","temp_column":"Neighborhood"})

In [39]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
NBH_df.shape

(103, 3)

In [40]:
NBH_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## SECTION B

### QUESTION 1: getting the latitude and the longitude coordinates of each neighborhood.

In [41]:
#!conda install -c conda-forge geocoder --yes

import geocoder # import geocoder

In [42]:
def get_geocoder(NBH_df):
    # initialize your variable to None
    Co_ordinates = None
    # loop until you get the coordinates
    while(Co_ordinates is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(NBH_df.strip()))
        Co_ordinates = g.latlng
        latitude = Co_ordinates[0]
        longitude = Co_ordinates[1]
    return latitude,longitude

In [43]:
# Add latitude and longitude to the Dataframe
NBH_df['Latitude'], NBH_df['Longitude'] = zip(*NBH_df['PostalCode'].apply(get_geocoder))
NBH_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### QUESTION 2: Generate map of Toronto showing neighbourhoods and how they cluster

In [44]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [45]:
#Map Toronto using geolocator
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.653963, -79.387207.


In [46]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(NBH_df['Latitude'], NBH_df['Longitude'], NBH_df['PostalCode'], NBH_df['Borough'], NBH_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(Toronto_map)
    
Toronto_map